TEXT

In [3]:
# IMPORTS
import numpy as np

In [8]:
file_path = 'instances_01_KP/low-dimensional/f1_l-d_kp_10_269'
data = np.loadtxt(file_path, dtype=int)
col_1 = data[:, 0]
col_2 = data[:, 1]

n_items = col_1[0]
capacity = col_2[0]

values = data[1:, 0]
weights = data[1:, 1]

# print("First Column:", col_1)
# print("Second Column:", col_2)
print("number of items:", n_items)
print("max weight:", capacity)
print("values:", values)
print("weights:", weights)

number of items: 10
max weight: 269
values: [55 10 47  5  4 50  8 61 85 87]
weights: [95  4 60 32 23 72 80 62 65 46]


In [9]:
def knapsack_fitness(solution, values, weights, capacity):
    total_weight = np.dot(solution, weights)
    total_value = np.dot(solution, values)
    if total_weight > capacity:
        return 0  # Invalid solution
    return total_value

In [10]:
def generate_random_solution(length):
    return np.random.randint(2, size=length)

In [30]:
test_sol = generate_random_solution(n_items)
test_sol_fitness = knapsack_fitness(test_sol, values, weights, capacity)
print(test_sol)
print(test_sol_fitness)

[0 0 0 1 1 1 1 0 0 1]
154
